In [ ]:
# from pathlib import Path

# from tokenizers import ByteLevelBPETokenizer

# paths = [str(x) for x in Path("/mnt/data/arxiv/source/tex2txt").glob("**/*.txt")]

# # # Initialize a tokenizer
# tokenizer = ByteLevelBPETokenizer()

# # Customize training
# tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
#     "<s>",
#     "<pad>",
#     "</s>",
#     "<unk>",
#     "<mask>",
# ], show_progress=True)
# # !mkdir arxivbert
# tokenizer.save_model("arxivbert")

In [3]:
# read the all the file names in the folder
# folder name: /mnt/data/arxiv/source/tex2txt
import os
import pandas as pd
from tqdm import tqdm
file_names = os.listdir('/mnt/data/arxiv/source/tex2txt')
file_names = file_names[:3000]
# merge all file into a txt
with open('/mnt/data/arxiv/source/all_file_bert1000.txt', 'w') as f:
    for file_name in tqdm(file_names):
        # read the file
        with open('/mnt/data/arxiv/source/tex2txt/' + file_name, 'r') as f1:
            text = f1.read()
            f.write(text)
            f.write("\n")

100%|██████████| 3000/3000 [00:00<00:00, 3244.53it/s] 


In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./arxivbert/vocab.json",
    "./arxivbert/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=128)

In [ ]:
tokenizer.encode("This is America")

In [ ]:
tokenizer.encode("This is bioinformatics").tokens


In [ ]:
!nvidia-smi

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=258,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./arxivbert", truncation=True, max_len=256)
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
model.num_parameters()

In [ ]:
from datasets import load_dataset

# dataset = load_dataset('text', data_files='/mnt/data/arxiv/source/all_file_bert.txt')

# # Tokenize the dataset
# def tokenize_function(examples):
#     return tokenizer(examples["text"], truncation=True, max_length=256)

# tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
# tokenized_dataset.save_to_disk('/mnt/data/arxiv/source/tokenized_dataset1000')

In [ ]:
from datasets import  load_from_disk
# Load the saved dataset from disk
tokenized_dataset = load_from_disk('/media/sdb/token_bert')

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_metric


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

training_args = TrainingArguments(
    output_dir="./arxivbert",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=48,
    save_steps=100,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset['train'],  # use the entire tokenized dataset
)

In [13]:

import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

trainer.train()

In [6]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="/media/sdb/arxiv_bert/all_year_model/2008",
    tokenizer="/media/sdb/arxiv_bert/all_year_model/wholewordtokenizer"
)
fill_mask("Classification is one of the most challenging tasks in sequence analysis since there are no explicit features and the dimensionality of the feature space is very high.")

In [17]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./aaaaaa",
    tokenizer="./aaaaaa"
)
fill_mask("This measure involves computing the Hamming distance between a substring of a sliding window of length $k$ and a $k$-mer $p \in P$, as shown in the following <mask>.")

[{'score': 0.27994075417518616,
  'token': 1996,
  'token_str': ' theorem',
  'sequence': 'This measure involves computing the Hamming distance between a substring of a sliding window of length $k$ and a $k$-mer $p \\in P$, as shown in the following theorem.'},
 {'score': 0.19669824838638306,
  'token': 4340,
  'token_str': ' lemma',
  'sequence': 'This measure involves computing the Hamming distance between a substring of a sliding window of length $k$ and a $k$-mer $p \\in P$, as shown in the following lemma.'},
 {'score': 0.13794490694999695,
  'token': 1300,
  'token_str': ' example',
  'sequence': 'This measure involves computing the Hamming distance between a substring of a sliding window of length $k$ and a $k$-mer $p \\in P$, as shown in the following example.'},
 {'score': 0.0771234780550003,
  'token': 5907,
  'token_str': ' proposition',
  'sequence': 'This measure involves computing the Hamming distance between a substring of a sliding window of length $k$ and a $k$-mer $p 